In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load both the train and test data set
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

In [3]:
# check on dimensions
print(train_set.shape, test_set.shape)

(1460, 81) (1459, 80)


In [4]:
# Combine the train and test data set before data wrangling
full_set = train_set.merge(test_set, how = 'outer')

In [5]:
# Remove the Id column
full_set.drop('Id', axis = 1, inplace = True)

In [ ]:
# Check on the histrogram for the train set
full_set.hist(bins = 50, figsize=(20,20))
plt.show()

In [ ]:
# use descrbie() to get more insight of the data distribution
full_set.describe().T

From the histagram and describe, we notice some outliers such as extremely large LotFontage. However, it might be possible for some super mansions. Let's keep them for now. One outlier that is wrong for sure is the 2207 in the GarageYrBlt. It is very likely that this value should be 2007 instead. Let's replace it

In [ ]:
full_set.GarageYrBlt.replace(2207,2007, inplace = True)

In [ ]:
full_set.describe().T

Now try to deal with the missing value by first checking which columns have the missing value

In [ ]:
# list down the missing value and its percentage 
missing = pd.concat([full_set.isna().sum(), full_set.isna().sum()/len(full_set)], axis = 1)
missing.columns = ['# of NaN','Percentage']
missing.sort_values('Percentage',ascending = False, inplace = True)
missing.head(40)

Some missing values actually mean that the house doesn't have that specific features. We can replace those missing values with None

In [ ]:
# The list of the meaningful missing values
None_list = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageFinish','GarageCond','GarageType','GarageType','GarageQual','BsmtFinType2','BsmtFinType1','BsmtExposure','BsmtCond','BsmtQual','MasVnrType']

In [ ]:
# Replace the missing value with None

for feature in None_list:
    full_set[feature].fillna('None', inplace = True)

In [ ]:
# Check out the remaining missing value
missing2 = pd.concat([full_set.isna().sum(), 100*full_set.isna().sum()/len(full_set)], axis = 1)
missing2.columns = ['# of missing values','Percentage']
missing2.sort_values('Percentage', ascending = False, inplace = True)
missing2.head(40)

Some missing values mean can be replaced by 0 to indicate that the house doesn't have that specific features

In [ ]:
# Replace the following missing value with 0
no_such_feature = ['GarageYrBlt','GarageArea','MasVnrArea','GarageCars','BsmtHalfBath','BsmtFullBath','BsmtFinSF2','BsmtFinSF1','TotalBsmtSF','BsmtUnfSF']

full_set.update(full_set[no_such_feature].fillna(0))

In [ ]:
full_set.isna().sum().sort_values(ascending = False).head(20)

For the missing values in LotFrontage, let's replace it with the mean value in the specific neighborhood the house belongs

In [ ]:
full_set['LotFrontage'] = full_set.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.mean()))

For the rest of the missing values, replace them with the most common value in its corresponding neighborhood

In [ ]:
# Replace the missing values in the list below with the most common value
common_list = ['MSZoning','Utilities','Functional','KitchenQual','Exterior2nd','Exterior1st','SaleType','Electrical']

for feature in common_list:
    full_set[feature] = full_set.groupby('Neighborhood')[feature].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
# Check the dataset again to make sure all the missing values have been dealt with
full_set.info()

In [ ]:
# It makes more sense to convert date related features into string dtype
date_list = ['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold','MoSold']
full_set[date_list] = full_set[date_list].astype(str)
